# Lab 1: First Steps with OpenAI API

**Week 1 - GenAI Introduction & Fundamentals**

**Provided by:** ADC ENGINEERING & CONSULTING LTD

## Objectives

In this lab, you will:
- Set up your OpenAI API environment
- Make your first API calls
- Understand API parameters (temperature, max_tokens, etc.)
- Experiment with different models
- Handle API responses and errors
- Monitor token usage and costs

## Prerequisites

- OpenAI API key
- Python 3.9+
- Basic Python knowledge

## Setup and Installation

In [ ]:
# Install required packages
!pip install openai python-dotenv tiktoken --quiet

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import tiktoken

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print("✓ Setup complete!")
print(f"✓ API Key configured: {'Yes' if os.getenv('OPENAI_API_KEY') else 'No'}")

## Part 1: Your First API Call

Let's make your first call to the OpenAI API using the Chat Completions endpoint.

In [ ]:
# Simple chat completion
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Hello! What can you help me with?"}
    ]
)

print("Response:")
print(response.choices[0].message.content)

### Understanding the Response Object

Let's explore what the API returns:

In [ ]:
# Examine the full response object
print("Full Response Object:")
print(f"ID: {response.id}")
print(f"Model: {response.model}")
print(f"Created: {response.created}")
print(f"\nUsage:")
print(f"  Prompt tokens: {response.usage.prompt_tokens}")
print(f"  Completion tokens: {response.usage.completion_tokens}")
print(f"  Total tokens: {response.usage.total_tokens}")
print(f"\nFinish reason: {response.choices[0].finish_reason}")

### Exercise 1.1: Make Your Own API Call

Create a function that sends a message and returns the response:

In [ ]:
def chat(message, model="gpt-3.5-turbo"):
    """
    Send a message to the OpenAI API and return the response.
    
    Args:
        message: The user's message
        model: The model to use (default: gpt-3.5-turbo)
    
    Returns:
        The assistant's response text
    """
    # TODO: Implement this function
    # Hint: Use client.chat.completions.create()
    
    pass

# Test your function
# result = chat("What is machine learning?")
# print(result)

## Part 2: Understanding API Parameters

The OpenAI API has several important parameters that control the model's behavior.

### Temperature

Temperature controls randomness:
- **0.0**: Deterministic, always picks the most likely token
- **0.7**: Balanced creativity and consistency
- **1.0+**: More random and creative

In [ ]:
def test_temperature(prompt, temperatures=[0.0, 0.5, 1.0]):
    """Test how temperature affects responses."""
    
    print(f"Prompt: {prompt}\n")
    print("="*80)
    
    for temp in temperatures:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=temp,
            max_tokens=100
        )
        
        print(f"\nTemperature: {temp}")
        print("-"*80)
        print(response.choices[0].message.content)
        print("="*80)

# Test with a creative prompt
creative_prompt = "Write the opening sentence of a science fiction story."
test_temperature(creative_prompt)

### Exercise 2.1: Find the Right Temperature

For each task below, what temperature would you use?

1. **Translation**: Translate "Hello, how are you?" to French
2. **Creative Writing**: Write a poem about autumn
3. **Factual Q&A**: What is the capital of France?
4. **Brainstorming**: Give me 5 unique business ideas

Test your hypotheses:

In [ ]:
# Test different tasks with different temperatures

tasks = {
    "translation": {
        "prompt": "Translate to French: 'Hello, how are you?'",
        "temperature": 0.0  # TODO: Adjust this
    },
    "creative": {
        "prompt": "Write a haiku about technology.",
        "temperature": 0.0  # TODO: Adjust this
    },
    "factual": {
        "prompt": "What is the speed of light?",
        "temperature": 0.0  # TODO: Adjust this
    },
    "brainstorm": {
        "prompt": "List 3 creative uses for a paperclip.",
        "temperature": 0.0  # TODO: Adjust this
    }
}

for task_name, task_config in tasks.items():
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": task_config["prompt"]}],
        temperature=task_config["temperature"],
        max_tokens=100
    )
    
    print(f"\n{task_name.upper()} (temp={task_config['temperature']}):")
    print(response.choices[0].message.content)
    print("-"*80)

### Max Tokens

Controls the maximum length of the response.

In [ ]:
def test_max_tokens(prompt, token_limits=[50, 100, 200]):
    """Test how max_tokens affects response length."""
    
    print(f"Prompt: {prompt}\n")
    
    for max_tokens in token_limits:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=0.7
        )
        
        content = response.choices[0].message.content
        actual_tokens = response.usage.completion_tokens
        
        print(f"\nMax tokens: {max_tokens}, Actual: {actual_tokens}")
        print(f"Finish reason: {response.choices[0].finish_reason}")
        print(f"Response: {content}")
        print("-"*80)

# Test
test_max_tokens("Explain what machine learning is.")

## Part 3: System Messages

System messages set the behavior and persona of the assistant.

In [ ]:
def chat_with_system(user_message, system_message):
    """Chat with a custom system message."""
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        temperature=0.7
    )
    
    return response.choices[0].message.content

# Test different system messages
user_msg = "What is Python?"

# Professional assistant
system_msg_1 = "You are a professional technical writer. Provide clear, concise explanations."
print("Professional Assistant:")
print(chat_with_system(user_msg, system_msg_1))
print("\n" + "="*80 + "\n")

# Fun assistant  
system_msg_2 = "You are a friendly teacher who explains things using fun analogies and examples."
print("Fun Teacher:")
print(chat_with_system(user_msg, system_msg_2))
print("\n" + "="*80 + "\n")

# Expert assistant
system_msg_3 = "You are a senior software engineer with 20 years of experience."
print("Expert Engineer:")
print(chat_with_system(user_msg, system_msg_3))

### Exercise 3.1: Design System Messages

Create system messages for these personas:

1. **Customer Support Agent**: Helpful, polite, empathetic
2. **Code Reviewer**: Critical, detail-oriented, constructive
3. **Creative Writer**: Imaginative, eloquent, poetic

In [ ]:
# TODO: Define system messages
customer_support_system = "You are..."
code_reviewer_system = "You are..."
creative_writer_system = "You are..."

# Test them
test_message = "The application crashed when I clicked submit."

print("Customer Support Response:")
# TODO: Call chat_with_system with customer_support_system
print("\n" + "="*80 + "\n")

# Try with code review
code_snippet = """
def calculate_total(items):
    total = 0
    for item in items:
        total = total + item
    return total
"""

print("Code Review:")
# TODO: Call chat_with_system with code_reviewer_system
print("\n" + "="*80 + "\n")

# Try with creative writer
topic = "A rainy day"
print("Creative Writing:")
# TODO: Call chat_with_system with creative_writer_system

## Part 4: Token Counting and Cost Estimation

Understanding token usage is crucial for managing costs.

In [ ]:
def count_tokens(text, model="gpt-3.5-turbo"):
    """Count the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Test token counting
texts = [
    "Hello!",
    "Hello, how are you doing today?",
    "The quick brown fox jumps over the lazy dog.",
    "In the beginning, the universe was created. This has made a lot of people very angry and been widely regarded as a bad move."
]

for text in texts:
    tokens = count_tokens(text)
    print(f"Text: {text[:50]}...")
    print(f"Tokens: {tokens}")
    print(f"Characters: {len(text)}")
    print(f"Ratio: {len(text)/tokens:.2f} chars/token")
    print("-"*80)

In [ ]:
def estimate_cost(prompt_tokens, completion_tokens, model="gpt-3.5-turbo"):
    """
    Estimate the cost of an API call.
    
    Pricing (as of 2024):
    - GPT-3.5-turbo: $0.0005/1K input, $0.0015/1K output
    - GPT-4: $0.03/1K input, $0.06/1K output
    """
    
    pricing = {
        "gpt-3.5-turbo": {"input": 0.0005, "output": 0.0015},
        "gpt-4": {"input": 0.03, "output": 0.06}
    }
    
    if model not in pricing:
        return None
    
    input_cost = (prompt_tokens / 1000) * pricing[model]["input"]
    output_cost = (completion_tokens / 1000) * pricing[model]["output"]
    total_cost = input_cost + output_cost
    
    return {
        "input_cost": input_cost,
        "output_cost": output_cost,
        "total_cost": total_cost,
        "total_tokens": prompt_tokens + completion_tokens
    }

# Example cost calculation
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Write a 200-word article about renewable energy."}
    ],
    max_tokens=300
)

cost_info = estimate_cost(
    response.usage.prompt_tokens,
    response.usage.completion_tokens,
    model="gpt-3.5-turbo"
)

print(f"Prompt tokens: {response.usage.prompt_tokens}")
print(f"Completion tokens: {response.usage.completion_tokens}")
print(f"Total tokens: {cost_info['total_tokens']}")
print(f"\nCost Breakdown:")
print(f"  Input cost: ${cost_info['input_cost']:.6f}")
print(f"  Output cost: ${cost_info['output_cost']:.6f}")
print(f"  Total cost: ${cost_info['total_cost']:.6f}")

### Exercise 4.1: Cost Comparison

Compare the cost of using GPT-3.5-turbo vs GPT-4 for the same task:

In [ ]:
prompt = "Explain quantum computing in simple terms."

for model in ["gpt-3.5-turbo", "gpt-4"]:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=200
    )
    
    cost_info = estimate_cost(
        response.usage.prompt_tokens,
        response.usage.completion_tokens,
        model=model
    )
    
    print(f"\n{model}:")
    print(f"  Tokens: {cost_info['total_tokens']}")
    print(f"  Cost: ${cost_info['total_cost']:.6f}")
    print(f"  Response preview: {response.choices[0].message.content[:100]}...")
    print("-"*80)

## Part 5: Error Handling

Production code needs robust error handling.

In [ ]:
from openai import OpenAIError, RateLimitError, APIError
import time

def safe_chat(message, max_retries=3, model="gpt-3.5-turbo"):
    """
    Make an API call with error handling and retries.
    """
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": message}],
                temperature=0.7
            )
            
            return {
                "success": True,
                "content": response.choices[0].message.content,
                "usage": response.usage
            }
            
        except RateLimitError:
            print(f"Rate limit hit. Waiting before retry {attempt + 1}/{max_retries}...")
            time.sleep(2 ** attempt)  # Exponential backoff
            
        except APIError as e:
            print(f"API error: {e}")
            if attempt == max_retries - 1:
                return {"success": False, "error": str(e)}
            time.sleep(1)
            
        except Exception as e:
            print(f"Unexpected error: {e}")
            return {"success": False, "error": str(e)}
    
    return {"success": False, "error": "Max retries exceeded"}

# Test the safe function
result = safe_chat("What is the meaning of life?")

if result["success"]:
    print("Response:", result["content"])
    print(f"Tokens used: {result['usage'].total_tokens}")
else:
    print("Error:", result["error"])

## Part 6: Multi-turn Conversations

Build a simple conversational interface:

In [ ]:
def conversation(system_message="You are a helpful assistant."):
    """
    Simple conversation loop.
    Type 'quit' to exit.
    """
    
    messages = [
        {"role": "system", "content": system_message}
    ]
    
    print("Chatbot ready! Type 'quit' to exit.\n")
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        
        if not user_input:
            continue
        
        # Add user message
        messages.append({"role": "user", "content": user_input})
        
        # Get response
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7
        )
        
        assistant_message = response.choices[0].message.content
        
        # Add assistant message
        messages.append({"role": "assistant", "content": assistant_message})
        
        print(f"\nAssistant: {assistant_message}\n")

# Uncomment to run the conversation
# conversation("You are a Python programming tutor.")

## Challenge Exercises

### Challenge 1: Smart Chatbot

Create a chatbot that:
1. Remembers conversation history
2. Tracks token usage
3. Warns when approaching token limits
4. Automatically summarizes old messages to save tokens

In [ ]:
class SmartChatbot:
    """
    A chatbot with conversation management.
    
    TODO: Implement the following methods:
    - __init__: Initialize with system message and token limit
    - add_message: Add a message to history
    - get_response: Get response from API
    - summarize_history: Summarize old messages when limit approached
    - get_token_count: Count tokens in conversation
    """
    
    def __init__(self, system_message, max_tokens=4000):
        self.system_message = system_message
        self.max_tokens = max_tokens
        self.messages = [{"role": "system", "content": system_message}]
    
    # TODO: Implement remaining methods
    
    pass

# Test your implementation
# bot = SmartChatbot("You are a helpful assistant.")
# response = bot.get_response("Hello!")
# print(response)

### Challenge 2: API Usage Logger

Create a logger that tracks:
- All API calls made
- Tokens used per call
- Cumulative costs
- Average response time

In [ ]:
import time
from datetime import datetime

class APILogger:
    """
    Log and analyze API usage.
    
    TODO: Implement methods to:
    - Log each API call
    - Calculate cumulative costs
    - Generate usage reports
    - Export to CSV
    """
    
    def __init__(self):
        self.calls = []
    
    def log_call(self, model, prompt_tokens, completion_tokens, duration):
        """Log an API call."""
        # TODO: Implement
        pass
    
    def get_report(self):
        """Generate usage report."""
        # TODO: Implement
        pass

# Usage example:
# logger = APILogger()
# 
# start = time.time()
# response = client.chat.completions.create(...)
# duration = time.time() - start
# 
# logger.log_call("gpt-3.5-turbo", prompt_tokens, completion_tokens, duration)
# print(logger.get_report())

### Challenge 3: Model Comparison Tool

Build a tool that:
1. Sends the same prompt to multiple models
2. Compares responses
3. Analyzes quality, speed, and cost
4. Recommends the best model for the task

In [ ]:
def compare_models(prompt, models=["gpt-3.5-turbo", "gpt-4"]):
    """
    Compare different models for the same task.
    
    TODO: Implement to:
    - Call each model with the same prompt
    - Measure response time
    - Calculate costs
    - Compare response quality (length, coherence, etc.)
    - Return comparison report
    """
    
    results = []
    
    # TODO: Implement comparison logic
    
    return results

# Test
# prompt = "Explain the theory of relativity in simple terms."
# comparison = compare_models(prompt)
# print(comparison)

## Summary

In this lab, you've learned:

1. ✅ How to make basic OpenAI API calls
2. ✅ Understanding key parameters (temperature, max_tokens)
3. ✅ Using system messages to control behavior
4. ✅ Counting tokens and estimating costs
5. ✅ Error handling and retries
6. ✅ Building multi-turn conversations

### Key Takeaways

- **Temperature**: Lower for factual, higher for creative tasks
- **System Messages**: Define the assistant's behavior and persona
- **Token Management**: Critical for cost control
- **Error Handling**: Always implement retries and proper error handling
- **Model Selection**: Choose based on task requirements and budget

### Next Steps

- Complete the challenge exercises
- Experiment with different parameters
- Build your own chatbot with custom features
- Move on to Lab 2: Text Generation Experiments